# Groningen Seismicity model
This notebook allows running the seismicity model developped at GMG for gas extraction from the Groningen gas field following Smith et al., 2022, Heimisson et al., 2022, and Meyer et al.,2022.

The most complete description of the model can be found in:

Smith, J. D., Heimisson, E. R., Bourne, S. J., & Avouac, J. P. (2022). Stress-based forecasting of induced seismicity with instantaneous earthquake failure functions: applications to the Groningen Gas Reservoir. Earth and Planetary Science Letters, 594, 117697.

---and---

Heimisson, E. R., Smith, J. D., Avouac, J. P., & Bourne, S. J. (2022). Coulomb threshold rate-and-state model for fault reactivation: application to induced seismicity at Groningen. Geophysical Journal International, 228(3), 2061-2072.


A full description of the needed libraries and tested practice on installation and running can be found in the file: Groningen_GMG_Seismicity_model_libraries.txt

A description of the integration of this module with the rest of the modelling workflow can be found in the file:
../README_GroningenModels_GMG.txt

## 1. Libraries and data set-up

### 1.1. Import Libraries

In [1]:
import pymc3 as pm
import numpy as np
from numpy.lib.stride_tricks import as_strided # to average arrays
import matplotlib.pylab as plt
import seaborn as sns
from math import *
import pandas as pd 
from matplotlib.patches import Circle, Wedge, Polygon
import scipy
import theano.tensor as tt
#the fancy axes
import matplotlib.gridspec as gridspec
# Gaussian Smoothing
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

### 1.2. Import functions

In [2]:
from util.plotting import cornerplot, rateplot
from util.failure import *
from util.forecasting import *

## 2. MCMC inference

### 2.1. Define the method to be used, and inference parameters

In [3]:
# Choose the failure function to be used
## TRS == Treshold Rate and State --> Heimisson et al., 2022
## RS == Classic Rate and State --> Dieterich., 1994
## ET == Extreme Threshold --> Bourne et al., 2017
## GF == Gaussian Failure --> Smith et al., 2022

Failure_function = 'TRS'
LogLikelihood = 'Gaussian'

if Failure_function =='TRS':
    method = ThresholdRateAndState_logSampled()
    labels = ['$r$ (evts/year)','$A \sigma$ (MPa)','$\Delta S_c$ (MPa)','$t_a$ (years)']
    LLK = LogLikelihood # log likelihood
    
elif Failure_function =='RS':
    method = ClassicRateAndState_logSampled() 
    labels = ['$r$ (evts/year)','$A \sigma$ (MPa)','$t_a$ (years)']
    LLK = LogLikelihood # log likelihood

elif Failure_function =='ET':
    method = ExtremeThreshold() 
    labels = ['$\beta_0$','$\beta_1$','$\beta_2$']
    LLK = LogLikelihood # log likelihood

elif Failure_function =='GF':
    method = GaussianFailure()
    labels = ['$\beta_0$','$\beta_1$','$\beta_2$']
    LLK = LogLikelihood # log likelihood
    
# Define parameters for the inversions
Num_samples = 5000 # samples for the MCMC inversion
Warmup_percentage = 20 # percentage burn-in samples
Num_chains = 10  #number of chains for the inversion
Training = [1992,2016] # Training period
Validation = [2016,2022] # Validation period

### 2.2. Prepare the data for the MCMC

In [4]:
# Load the Coulomb Stress data and average yearly
dC_raw = np.load('../Simulation_results/max_coulomb_stresses.npy',allow_pickle=True).item()['-10m'] # use item() because it is an object array that can contain calculations at different depths
# Smooth it to 3km 
kernel = Gaussian2DKernel(6)
dC = np.array([convolve(dC_raw[:,:,ii],kernel,nan_treatment='fill') for ii in range(dC_raw.shape[-1])]).transpose((1, 2, 0))

# Average yearly
dC_y ={} # Disctionnary to store the yearly data
dC_y['Coulomb'] = np.nanmean(as_strided(dC, shape=(dC.shape[0], dC.shape[1], int(dC.shape[-1]/12), 12),strides=(dC.strides[0], dC.strides[1], dC.strides[2]*12, dC.strides[2])),axis=3)
dC_y['Time']    = np.arange(0,dC_y['Coulomb'].shape[-1])*(1)+1956.0

# Load the seismicity catalog
catalog_file = '../Reservoir_data/Seismic_catalog/KNMI_CAT_1991-12to2023-01_InsideReservoir.csv'

# Define the completion magnitude to use
Mc =1.5

### 2.3. Run MCMC

In [ ]:
# Run the MCMC inversion
# Create the forecasting object
SF = SeismicityForecasting(method,catalog_file,dC_y,num_samples=Num_samples,warmup_percentage=Warmup_percentage,num_chains=Num_chains,Mc=Mc)# load the forecasting model and give the parameters)
# Run the inference
SF.run(training=Training,validation=Validation,LLK=LLK)

========= Training - Running MCMC =========


## 3. Plot the results

### 3.1 The corner plots for best parameters

In [ ]:
# Plot the data
File_cornerplot = f'Figures/cornerplot_method={method}.png'
cornerplot(SF,File_cornerplot)

### 3.2. The rate plots for temporal evolution

In [ ]:
File_rateplot = f'Figures/rateplot_method={method}.png'
rateplot(SF,File_rateplot)